In [1]:
using FFTW

In [2]:
using Plots
using Dates
using BenchmarkTools

# The Kuramoto-Sivashinsky Equation: Numerics

The Kuramoto-Sivashinsky equation is a PDE that was originally formulated to describe the [fronts of laminar flames](https://www.jstor.org/stable/2100687?seq=1) by G.I. Sivashinsky. The PDE has two forms. In one-dimension, it may be written in *derivative form*:
$$u_t + u u_x + u_{xx} + u_{xxxx} = 0$$
or in *integral form*, which relies on the change of variables $u = v_t$:
$$v_t + \frac 1 2 v_x^2 + v_{xx} + v_{xxxx} = 0$$
In multiple dimensions, it takes the form
$$u_t + a \lvert \nabla u \rvert^2 + \Delta u + \Delta^2 u = 0$$
Where $a$ is often set to $\frac 1 2$

# One-Dimensional Solver
We will use Galerkin reduction, representing $v$ as a partial fourier series
$$v \approx v^N = \sum_{k = -N}^N c_k(t) \exp {\frac{2 \pi i k x}{L}}$$
where the solutions are kept real by enforcing $c_{-k} = c_k^*$. We will also set $L = 2 \pi K$. Then performing Galerkin Reduction, we have the following system of ODEs:
$$\frac{d}{dt}c_m(t) = \left(\frac{m^2}{K^2} - \frac{m^4}{K^4}\right) c_m(t) + \frac{a}{K^2}\sum_{k = -N}^{N} k(m - k) c_k(t)c_{m - k}(t)$$
wherever the summand is defined. We can integrate this using our exponential second-order method.

## Integrator

In [3]:
function Expo_Heun(T, λ, c₀, g, h, N)
    c = zeros(Complex, T, N+1)
    c[1, :] = c₀
    φ(m) = (ℯ^m - 1) / m * (m != 0) + 1 * (m == 0)
    φ₀(m) = 1 / m^2 * (ℯ^m - m - 1) * (m != 0) + 1/2 * (m == 0)
    φ₁(m) = 1 / m^2 * (m * ℯ^m - ℯ^m + 1) * (m != 0) + 1/2 * (m == 0)
    for i in 1:T-1
        ### Euler Steps
        c_EULER = ℯ.^(-h * λ) .* (c[i, :] + h * φ.(h * λ) .* g(c[i, :]))

        ### In-class CN Steps
        c[i+1, :] = ℯ.^(-h * λ) .* (c[i, :] + h * φ₀.(h * λ) .* g(c[i, :]) + h .* φ₁.(h * λ) .* g(c_EULER))
    end
    return c
end

Expo_Heun (generic function with 1 method)

## Nonlinear Term

In [4]:
function compute_convolution(c, m, N, K)
    sum = 0.0im
    for k in -N:N
        if abs(m-k) ≤ N
            ck = k < 0 ? conj(c[abs(k)+1]) : c[abs(k)+1]
            cm_k = (m-k) < 0 ? conj(c[abs(m-k)+1]) : c[abs(m-k)+1]
            sum += im*k*ck*cm_k
        end
    end
    return sum/K
end

function compute_convolution_int(c, m, N, K)
    sum = 0.0im
    for k in -N:N
        if abs(m-k) ≤ N
            ck = k < 0 ? conj(c[abs(k)+1]) : c[abs(k)+1]
            cm_k = (m-k) < 0 ? conj(c[abs(m-k)+1]) : c[abs(m-k)+1]
            sum += k*(m-k)*ck*cm_k
        end
    end
    return sum/K^2  # Added the 1/K^2 factor
end

function conv_all(c, N, K)
    out = zeros(Complex, N+1)
    for m = 0:N
        out[m + 1] = compute_convolution(c, m, N, K)
    end
    return out
end

function conv_all_int(c, N, K)
    out = zeros(Complex, N+1)
    for m = 0:N
        out[m + 1] = compute_convolution_int(c, m, N, K)
    end
    return out
end

conv_all_int (generic function with 1 method)

## Frequency Domain and Spatial Domain

In [5]:
function frequency_to_space(c, K)
    N = size(c)[1] - 1
    return x -> begin
        u = c[1]  # k=0 term
        for k in 1:N
            u += c[k+1]*exp(im*k*x / K) + conj(c[k+1])*exp(-im*k*x / K)
        end
        return real(u)
    end
end

frequency_to_space (generic function with 1 method)

## Kuramoto-Sivashinsky Test

In [24]:
N = 150
h = 1/10
a = 1
K = 40
L = 2 * π * K
g(c) = - a .* conv_all(c, N, K)
λ(ε, N) = [(-m^2 / K^2 + ε * m^4 / K^4) for m in 0:N]

T = 1000

c₀ = [rand(Complex{Float64})/(1 + k) * .05 for k in 0:N]
c₀[1] = 0

x = range(0, L, length = 500)

c = Expo_Heun(T, λ(1, N), c₀, g, h, N)

st = mapreduce(permutedims, vcat, ([frequency_to_space(c[i,:], K).(x) for i in 1:T]))

anim = @animate for i in 1:T
    u = frequency_to_space(c[i,:], K)
    plot(x, u, ylims = (-3,3))
end
# display(gif(anim, "animation.gif", fps=40))

Animation("/var/folders/qm/0fw8nlh915s8bk_yfrn_y8vm0000gp/T/jl_COtXiW", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000991.png", "000992.png", "000993.png", "000994.png", "000995.png", "000996.png", "000997.png", "000998.png", "000999.png", "001000.png"])

In [27]:
N = 40
h = 1/15
a = 1/2
K = 15
L = 2 * π * K
g(c) = + a .* conv_all_int(c, N, K)
λ(ε, N) = [(-m^2 / K^2 + ε * m^4 / K^4) for m in 0:N]

T = 1000

c₀ = [rand(Complex{Float64})/(1 + k) * .05 for k in 0:N]
c₀[1] = 0

x = range(0, L, length = 500)

c = Expo_Heun(T, λ(1, N), c₀, g, h, N)

st = mapreduce(permutedims, vcat, ([frequency_to_space(c[i,:], K).(x) for i in 1:T]))

anim = @animate for i in 1:T
    u = frequency_to_space(c[i,:], K)
    plot(x, u)
end
# display(gif(anim, "animation.gif", fps=40))

Animation("/var/folders/qm/0fw8nlh915s8bk_yfrn_y8vm0000gp/T/jl_87x6zi", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000991.png", "000992.png", "000993.png", "000994.png", "000995.png", "000996.png", "000997.png", "000998.png", "000999.png", "001000.png"])

In [26]:
p=heatmap(x, h:h:T*h, st,
    size=(1000, 600),
    camera = ((135), 40),
    zlims=(-6,6),
    # legend=:none,
    title="Kuramoto-Sivashinsky chaos (derivative form)",
    xticks=0:50:L,
    xlabel="\$x\$",
    ylabel="\$t\$"
)
savefig(p, "plots/ks1d2")

"/Users/azul/Documents/Assignments/APMA 2550/Final Exam/kuramoto-sivashinsky/plots/ks1d1.png"

In [37]:
p=surface(x, h:h:T*h, st,
    size=(1000, 600),
    camera = ((135), 40),
    # zlims=(-6,1),
    # legend=:none,
    title="Kuramoto-Sivashinsky chaos (derivative form)",
    xticks=0:50:L,
    xlabel="\$x\$",
    ylabel="\$t\$"
)
savefig(p, "plots/ks1d2")

"/Users/azul/Documents/Assignments/APMA 2550/Final Exam/kuramoto-sivashinsky/plots/ks1d2.png"

In [28]:
c_avg = copy(c)
c_avg[:,1] .= 0
st2 = mapreduce(permutedims, vcat, ([frequency_to_space(c_avg[i,:], K).(x) for i in 1:T]));

In [38]:
p = heatmap(x, h:h:T*h, st2,
    size=(1000, 600),
    camera = ((135), 40),
    zlims=(-6,6),
    legend=:none,
    title="Integral Form",
    # xticks=0:50:L,
    xlabel="\$x\$",
    ylabel="\$t\$"
)
savefig(p, "plots/ks1d3")

"/Users/azul/Documents/Assignments/APMA 2550/Final Exam/kuramoto-sivashinsky/plots/ks1d3.png"